# Cereal Time Killers: Deep learning model to predict emotional states based on EEG data.

[TODO: Introduction and description of project]

## Setup

### Install dependencies

In [9]:
import time
import torch
import pathlib
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.signal import spectrogram

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset,DataLoader, TensorDataset

from tqdm.notebook import tqdm, trange
from PIL import Image 

# Data processing

In [10]:
def get_specgram(dir_to_GAMEEMO, patient, game,winlen=None,stride=1):
    # Reading from the csv data set (can do matlab as well) using pandas. 

    #Patient = 'S01' #The Patient
    #Game = 'G1' #The game

    #You can also just paste in the Directory of the csv file - on windows you may have to change the slash direction
    DirComb = f'{dir_to_GAMEEMO}/({patient})/Preprocessed EEG Data/.csv format/{patient}{game}AllChannels.csv'

    df=pd.read_csv(DirComb, sep=',',header=None)
    d = np.array(df) #Switching from pandas to numpy array as this might be more comfortable for people
    d = np.delete(d,0,0) # Deleting header
    d = np.delete(d,-1,1).astype(float) #and erroneous last column
    
    full_spec = []
    for idx, d2 in enumerate(d.T):
        _, _, Sxx = spectrogram(d2,fs=120)
        full_spec.append(Sxx)
        
    #DIMENSIONS OF FULL_SPEC WITHOUT WINDOWING
    #DIMENSION 1: CHANNELS  (DEFAULT=14) - MIGHT CHANGE (SO NOT REALLY DEFAULT BUT OK)
    #DIMENSION 2: FREQUENCY (DEFAULT=129)
    #DIMENSION 3: TIME      (DEFAULT=170) - MIGHT CHANGE AS WELL OK - WE ARE WORKING ON IT
    
    full_spec = np.vstack([full_spec])

    if(winlen==None):
        return full_spec
    
    i = 0
    
    
    full_spec_wind = []
    while i*stride+winlen<full_spec.shape[-1]:
        full_spec_wind.append(full_spec[:,:,stride*i:i*stride+winlen])
        i+=1    
    
    #DIMENSIONS OF FULL_SPEC WITH WINDOWING    (FULL_SPEC_WIND) 
    #DIMENSION 1: CHANNELS  (DEFAULT=14) - MIGHT CHANGE (SO NOT REALLY DEFAULT BUT OK)
    #DIMENSION 2: FREQUENCY (DEFAULT=129)
    #DIMENSION 3: TIME      (NO DEFAULT - SORRY)
    #DIMENSION 4: WINDOWS   (NO DEFAULT - SORRY)
    
    full_spec_wind = np.array(full_spec_wind)
    full_spec_wind = np.moveaxis(full_spec_wind,0,-1)
    return full_spec_wind

In [11]:
basedir = '/Users/maximilianeggl/Dropbox/PostDoc/NeuroMatch/'
labeldir = f'{basedir}GameLabels.csv' #Path to GAMEEMO
gamedir = f'{basedir}GAMEEMO/'

full_spec = get_specgram(gamedir, 'S01', 'G1',25,1)

In [166]:
labels_df = pd.read_csv(labeldir)

labels_df['full_specgram_1'] = [[]] * len(labels_df)
for idx in range(len(labels_df)):
    game = f'G{int(idx % 4)+1}'
    subject = f'S{int(idx/4)+1:02d}'
    labels_df['full_specgram_1'][idx] = torch.tensor(get_specgram(gamedir, subject, game)).double()

<ipython-input-166-ea7a13fb5f49>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_df['full_specgram_1'][idx] = torch.tensor(get_specgram(gamedir, subject, game)).double()


In [167]:
class CerealTimeKillersDataset(Dataset):
    """Spectrogram dataset."""

    def __init__(self, df):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ori_dataframe = df

    def __len__(self):
        return len(self.ori_dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        spectrogram = self.ori_dataframe.iloc[idx, 14]
        labels = self.ori_dataframe.iloc[idx, :14]
        labels = torch.tensor([labels[8:]])
        sample = (spectrogram,labels)

        return sample

In [168]:
final_dataset = CerealTimeKillersDataset(labels_df)
train_loader,test_loader,val_loader = torch.utils.data.random_split(final_dataset, [40,40,28])
train_loader = DataLoader(train_loader, batch_size=4, shuffle=True,
                        num_workers=0)
test_loader = DataLoader(test_loader, batch_size=4, shuffle=True,
                        num_workers=0)
val_loader = DataLoader(val_loader, batch_size=4, shuffle=True,
                        num_workers=0)

In [169]:
train_loader.dataset[0][1]

tensor([[3, 7, 5, 5, 4, 5]])

# Implementation model

# Model: Convolutional Neural Network

In [192]:
class CTK_Net(nn.Module):
  def __init__(self, out_size, img_shape):
    """
    INPUT:
      out_size : size of the output. It should match the number of labels in the dataset.
      img_shape : list len 2
    """
    super(CTK_Net, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=14, out_channels=32, kernel_size=3)
    #self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
    self.pool = nn.MaxPool2d(kernel_size=2)
    # WARNING: change the hardcoded values in fc1_input_size if you change the architecture!!!
    fc1_input_size = int(32 * np.prod((np.array(img_shape) - 3 + 1 - 3 + 1) / 2))
    self.fc1 = nn.Linear(in_features=fc1_input_size, out_features=128)
    self.drop1 = nn.Dropout(.5)
    self.fc2 = nn.Linear(in_features=128, out_features=out_size)
    

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.sigmoid(x)
    return x

In [193]:
args = {
    'epochs': 150,
    'lr': 5e-3,
    'momentum': 0.99,
    'device': 'cpu',
}

TestNet = CTK_Net(5,np.array((149,170)))

In [194]:
def train(args, model, train_loader, optimizer=None, criterion=F.nll_loss):
  model.train()

  criterion = nn.CrossEntropyLoss() #Change as desired
  optimizer = torch.optim.SGD(model.parameters(), 
                            lr=args['lr'])
  for epoch in range(args['epochs']):
    with tqdm(train_loader, unit='batch') as tepoch:
      for data, target in tepoch:
        data, target = data.type(torch.float).to(device), target.to(device)
        optimizer.zero_grad() 
        output = model(data)
        loss = criterion(output, target)
        
        # if reg_function1 is None:
        #   loss = criterion(output, target)
        # elif reg_function2 is None:
        #   loss = criterion(output, target)+args['lambda']*reg_function1(model)
        # else:
        #   loss = criterion(output, target) + args['lambda1']*reg_function1(model) + args['lambda2']*reg_function2(model)

        loss.backward()
        optimizer.step()
        tepoch.set_postfix(loss=loss.item())
        time.sleep(0.1)


def test(model, device, data_loader):
  model.eval()
  correct = 0
  total = 0
  for data in data_loader:
    inputs, labels = data
    inputs = inputs.to(device).float()
    labels = labels.to(device).long()

    outputs = model(inp'uts)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  acc = 100 * correct / total
  return acc

SyntaxError: EOL while scanning string literal (<ipython-input-194-5c50dcf4bb46>, line 38)

In [ ]:
train(args,TestNet,train_loader)

  0%|          | 0/10 [00:00<?, ?batch/s]

> <ipython-input-192-73c42ae6b5f3>(22)forward()
     20   def forward(self, x):
     21     breakpoint()
---> 22     x = self.conv1(x)
     23     x = F.relu(x)
     24     x = torch.flatten(x, 1)

ipdb> x = x[0]
ipdb> x.shape
torch.Size([14, 129, 170])
ipdb> n
RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 14, 3, 3], but got 3-dimensional input of size [14, 129, 170] instead
> <ipython-input-192-73c42ae6b5f3>(22)forward()
     20   def forward(self, x):
     21     breakpoint()
---> 22     x = self.conv1(x)
     23     x = F.relu(x)
     24     x = torch.flatten(x, 1)



In [140]:
device='cpu'